<a href="https://colab.research.google.com/github/mkorob/DementiaDataSynthEval/blob/main/Alzheimer's_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
import re

In [ ]:
import time

In [ ]:
#link of the webpage
link = "https://dementiadiaries.org/our-diarists/"
t = requests.get(link)
soup = BeautifulSoup(t.text)

#we don't need to iterate through pages because we want to take first X authors anyways
max_diarists = 10
max_posts_per_diarist = 20
max_attempts_per_diarist = 40
min_length_text = 200

df_out = pd.DataFrame()

def get_all_posts_out(df_out, url_diarist, page, counter_diarist_items, max_counter_diarists):
  print(f"{url_diarist}/page/{page}")
  soup_diarist = BeautifulSoup(requests.get(f"{url_diarist}/page/{page}").text)
  diarist_items = soup_diarist.find_all("div", class_= "uabb-blog-post-content")
  #counter_diarist_items = 0
  for diarist_item in diarist_items:
    diarist_item_link = diarist_item.find("a")['href']
    time.sleep(3)
    soup_diary_entry = BeautifulSoup(requests.get(diarist_item_link).text)
    date = soup_diary_entry.find("span", class_= "fl-post-info-date").text
    paragraphs = soup_diary_entry.find_all("div", class_ = "fl-html")
    for paragraph in paragraphs:
      ps = paragraph.find_all("p")
      #some of these body types are not text, there should be only one item here
      if len(ps) > 1:
        text_diary_out = ""
        for p in ps:
          text_diary_out += p.text.strip()
        #let's add it anyways
        word_count = len(text_diary_out.split())
        entry_date = pd.DataFrame(data =  {'author_ID': [counter_diarists+1], 'text': [text_diary_out], 'date': [date], 'word_count': [word_count]})
        df_out = pd.concat([df_out, entry_date])
        if len(text_diary_out.split()) >= min_length_text:
          counter_diarist_items += 1
    #make sure to not keep iterating over a person who does not write 200 wors, because that will not be representative
    max_counter_diarists += 1
    #if maximum number reached
    if (counter_diarist_items > max_posts_per_diarist) or (max_counter_diarists > max_attempts_per_diarist):
      return df_out, counter_diarist_items, max_counter_diarists
  #otherwise return when done
  return df_out, counter_diarist_items, max_counter_diarists

In [ ]:
counter_diarists = 0
diarists = soup.find_all("div", class_="diarists-column")
for diarist in diarists:
  print("one")
  #check how many posts they post
  count = diarist.find("span", class_= "diaristpostcount").text
  if int(re.findall(r'\d+', count)[0]) > max_posts_per_diarist:
    url_diarist = diarist.find_all("a", href = True)[0]['href']
    time.sleep(3)
    page = 0
    #Page 1
    counter_diarist_items = 0
    max_counter_diarists = 0
    while page < 3:
      page +=1
      try:
        df_out, counter_diarist_items, max_counter_diarists = get_all_posts_out(df_out, url_diarist, page, counter_diarist_items, max_counter_diarists)
        #if maximum reached or maximum attempts , break loop
        if (counter_diarist_items > max_posts_per_diarist) or (max_counter_diarists > max_attempts_per_diarist):
          break
      except:
        #if ran out pages, go to the next author
        print(f"Unknown error occured parsing {url_diarist} page {page}")
        break

    #one more diarist out
    counter_diarists += 1
    if (counter_diarists == max_diarists):
      break

one
https://dementiadiaries.org/entry/diarist/agnes-houston//page/1
https://dementiadiaries.org/entry/diarist/agnes-houston//page/2
https://dementiadiaries.org/entry/diarist/agnes-houston//page/3
one
one
one
one
one
https://dementiadiaries.org/entry/diarist/anne-macdonald//page/1
https://dementiadiaries.org/entry/diarist/anne-macdonald//page/2
https://dementiadiaries.org/entry/diarist/anne-macdonald//page/3
one
one
one
one
one
one
https://dementiadiaries.org/entry/diarist/carol-fordyce//page/1
https://dementiadiaries.org/entry/diarist/carol-fordyce//page/2
https://dementiadiaries.org/entry/diarist/carol-fordyce//page/3
one
one
one
one
one
https://dementiadiaries.org/entry/diarist/chris-norris//page/1
https://dementiadiaries.org/entry/diarist/chris-norris//page/2
https://dementiadiaries.org/entry/diarist/chris-norris//page/3
one
one
one
https://dementiadiaries.org/entry/diarist/christine-maddocks//page/1
https://dementiadiaries.org/entry/diarist/christine-maddocks//page/2
https://dement

In [ ]:
df_out

,author_ID,text,date,word_count
0,1,Good morning everybody it’s Agnes from Coatbri...,Posted on 14 May 2021,272
0,1,"Good morning everybody, it’s Agnes from Coatbr...",Posted on 30 April 2021,157
0,1,"Good morning everybody, it’s Agnes from Coatbr...",Posted on 25 March 2021,203
0,1,Good morning everybody. It’s the weekend. I be...,Posted on 26 February 2021,269
0,1,Hi everyone. It’s Agnes from Coatbridge calli...,Posted on 19 February 2021,294
...,...,...,...,...
0,10,"Hi, it’s Gail and this morning I’m going to re...",Posted on 15 January 2021,601
0,10,"Hi, it’s Gail here this morning and it’s a ver...",Posted on 08 January 2021,598
0,10,"Hi, it’s Gail and I thought I’d share with you...",Posted on 01 December 2020,51
0,10,"Hi, it’s Gail.I just thought I’d share with yo...",Posted on 26 November 2020,307


In [ ]:
df_out.to_csv("dementia_diaries_final.csv")

In [ ]:
df_out_filtered = df_out[df_out['word_count'] > 200]

In [ ]:
df_out_filtered.groupby('author_ID').count()

,text,date,word_count
author_ID,,,
1,21,21,21
2,6,6,6
3,18,18,18
4,3,3,3
5,14,14,14
6,18,18,18
8,12,12,12
9,12,12,12
10,17,17,17
